<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-04-16 09:55:58--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  5.88MB/s    in 14m 9s  

2023-04-16 10:10:10 (1.63 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [16]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [17]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [18]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [19]:
# method most_simmilar
for idx, sim in enumerate(wv_embeddings.most_similar('dog', topn=30)):
    print(idx + 1, sim)

1 ('animal', 0.8564180135726929)
2 ('dogs', 0.7880866527557373)
3 ('mammal', 0.7623804211616516)
4 ('cats', 0.7621253728866577)
5 ('animals', 0.760793924331665)
6 ('feline', 0.7392398715019226)
7 ('bird', 0.7315488457679749)
8 ('animal1', 0.7219215631484985)
9 ('doggy', 0.7213349342346191)
10 ('labrador', 0.7209131717681885)
11 ('canine', 0.7209057211875916)
12 ('meow', 0.7185295820236206)
13 ('cow', 0.7080444693565369)
14 ('dog2', 0.7057910561561584)
15 ('woof', 0.705061137676239)
16 ('dog1', 0.7038840651512146)
17 ('dog3', 0.701882004737854)
18 ('penguin', 0.697029173374176)
19 ('bulldog', 0.6940488815307617)
20 ('mammals', 0.6931389570236206)
21 ('bark', 0.6913798451423645)
22 ('fruit', 0.6892251968383789)
23 ('reptile', 0.6891209483146667)
24 ('furry', 0.6863499283790588)
25 ('carnivore', 0.6862949728965759)
26 ('cat', 0.6852341294288635)
27 ('horse', 0.6833381652832031)
28 ('kitten', 0.6820152997970581)
29 ('sheep', 0.6802570223808289)
30 ('chihuahua', 0.6791757941246033)


**Ответ:** В `top-5` входит слово `cats` (на 4 месте), а слово `cat` попало только на 26 место.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

Зададим здесь три различных токенайзера, которые далее будем использовать для сравнения влияния различных подходов к токенизации на качество ранжирования.

In [78]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=22da21f7025e70468268943021f7289ed08087694d2a9b5a75adfd23c1acfec2
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [79]:
import numpy as np
import pymorphy2
import re
import spacy

from nltk.tokenize import WordPunctTokenizer

In [63]:
class MyTokenizer:
    def tokenize(self, text):
        return re.findall('\w+', text)

In [58]:
nlp = spacy.load('en_core_web_sm')

class SpacyTokenizer:
    def tokenize(self, text):
        doc = nlp(text)
        return [token.lemma_ for token in doc]

In [82]:
class Pymorphy2Tokenizer:
    def __init__(self):
        self.normalizer = pymorphy2.MorphAnalyzer()
    def tokenize(self, text):
        doc = list(map(lambda x: self.normalizer.parse(x), text.split(' ')))
        return [res[0].normal_form for res in doc] 

In [84]:
word_punct_tokenizer = WordPunctTokenizer()
spacy_tokenizer = SpacyTokenizer()
my_tokenizer = MyTokenizer()
pymorph_tokenizer = Pymorphy2Tokenizer()

tokenizers = [pymorph_tokenizer, word_punct_tokenizer, my_tokenizer]

In [22]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    tokens = tokenizer.tokenize(question.lower())
    token_embeddings = []
    for token in tokens:
        if token in embeddings:
            token_embeddings.append(embeddings[token])
    
    if len(token_embeddings) > 0:
        return np.mean(token_embeddings, axis=0)
    else:
        return np.zeros(dim, dtype='float32')

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [60]:
prompt = 'I love neural networks'
embedding = question_to_vec(prompt, wv_embeddings, my_tokenizer)
print(embedding[2])
print(round(embedding[2], 2))

-1.2854122
-1.29


**Ответ:** С округлением `-1.29`

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

**Ответ:** Максимум метрики `Hits@47` достигается когда дубликаты вопросов расположены на позициях до 47 включительно, и равняется `1`. Минимум функции `DCG@1` достигается в случае, если дубликаты вопросов не попадают на первую позицию, и равняется `0`. Соответственно, максимум разницы `Hits@47 - DCG@1` равняется `1` и достигается, когда все дубликаты находятся на позициях со `2` по `47` включительно.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [61]:
import math

print(f'Ответ: DCG@10 = {round(1 / math.log2(1 + 9), 1)}')

Ответ: DCG@10 = 0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [25]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """

    hits_value = sum([int(rank <= k) for rank in dup_ranks]) / len(dup_ranks)
    return hits_value    

In [26]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = sum([int(rank <= k) / math.log2(1 + rank) for rank in dup_ranks]) / len(dup_ranks)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [27]:
import pandas as pd

In [28]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [29]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
warning [stackoverflow_similar_questions.zip]:  1048576 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  1048576
  (attempting to re-compensate)
error: invalid zip file with overlapped components (possible zip bomb)


Кажется разархивирование данного архива в Colab не работает. Рекомендую скачать и распаковать его вручную, а затем подгрузить данные в Colab.

Считайте данные.

In [30]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [32]:
validation_data = read_corpus('validation.tsv')

Кол-во строк

In [33]:
len(validation_data)

3760

Размер нескольких первых строк

In [34]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [45]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    ranking = []
    question_embedding = question_to_vec(question, embeddings, tokenizer).reshape(1, -1)

    for idx, candidate in enumerate(candidates):
        candidate_embedding = question_to_vec(candidate, embeddings, tokenizer).reshape(1, -1)
        similarity = cosine_similarity(question_embedding, candidate_embedding)
        ranking.append((similarity, idx, candidate))

    ranking.sort(key=lambda x: x[0], reverse=True)
    return [(x[1], x[2]) for x in ranking]


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [43]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [68]:
for tokenizer in tokenizers:
    print(type(tokenizer).__name__)
    for idx, (question, q_candidates) in enumerate(zip(questions, candidates)):
            ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
            print(f'{idx + 1} experiment: {[r[0] for r in ranks]}')
    print()

WordPunctTokenizer
1 experiment: [1, 0, 2]
2 experiment: [0, 2, 1]

SpacyTokenizer
1 experiment: [1, 0, 2]
2 experiment: [0, 2, 1]

MyTokenizer
1 experiment: [1, 0, 2]
2 experiment: [0, 2, 1]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [49]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'),
            (2, 'select2 not displaying search results'),
            (1, 'WPF- How to update the changes in list item of a list')]]

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ:** `021`

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

Ускорим вычисления и возьмем 100 примеров

In [75]:
from tqdm.notebook import tqdm

In [85]:
wv_ranking = {}
max_validation_examples = 100
for tokenizer in tokenizers:
    tokenizer_name = type(tokenizer).__name__
    print(f'starting work with tokenizer: {tokenizer_name}')
    wv_ranking[tokenizer_name] = []
    for i, line in enumerate(tqdm(validation_data[:max_validation_examples])):
        q, *ex = line
        ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
        wv_ranking[tokenizer_name].append([r[0] for r in ranks].index(0) + 1)

starting work with tokenizer: Pymorphy2Tokenizer


  0%|          | 0/100 [00:00<?, ?it/s]

starting work with tokenizer: WordPunctTokenizer


  0%|          | 0/100 [00:00<?, ?it/s]

starting work with tokenizer: MyTokenizer


  0%|          | 0/100 [00:00<?, ?it/s]

In [86]:
for tokenizer in tokenizers:
    tokenizer_name = type(tokenizer).__name__
    print(f'Metrics for tokenizer: {tokenizer_name}')
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking[tokenizer_name], k), k, hits_count(wv_ranking[tokenizer_name], k)))
    print()

Metrics for tokenizer: Pymorphy2Tokenizer


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.380 | Hits@   1: 0.380
DCG@   5: 0.463 | Hits@   5: 0.540
DCG@  10: 0.482 | Hits@  10: 0.600
DCG@ 100: 0.526 | Hits@ 100: 0.830
DCG@ 500: 0.546 | Hits@ 500: 0.980
DCG@1000: 0.548 | Hits@1000: 1.000

Metrics for tokenizer: WordPunctTokenizer


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.390 | Hits@   1: 0.390
DCG@   5: 0.490 | Hits@   5: 0.580
DCG@  10: 0.524 | Hits@  10: 0.680
DCG@ 100: 0.569 | Hits@ 100: 0.900
DCG@ 500: 0.581 | Hits@ 500: 0.990
DCG@1000: 0.582 | Hits@1000: 1.000

Metrics for tokenizer: MyTokenizer


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.390 | Hits@   1: 0.390
DCG@   5: 0.490 | Hits@   5: 0.590
DCG@  10: 0.522 | Hits@  10: 0.690
DCG@ 100: 0.567 | Hits@ 100: 0.910
DCG@ 500: 0.577 | Hits@ 500: 0.980
DCG@1000: 0.579 | Hits@1000: 1.000



### Эмбеддинги, обученные на корпусе похожих вопросов

In [87]:
train_data = read_corpus('train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

На предобученных эмбеддингах наилучшее качество показал токенизатор WordPunctTokenizer. Возьмем его и обучим Word2Vec с min_count=5, как указано в задании, и алгоритмом Skip-Gram (он лучше справляется с небольшими датасетами и моделированием редко встречающихся слов — так как мы имеем дело с датасетом со StackOverFlow, это кажется разумным выбором), и обучим Word2Vec с разными размерами окна — пусть это будут четные числа до 10 включительно.

In [88]:
from gensim.models import Word2Vec

In [93]:
words = [word_punct_tokenizer.tokenize(sample[0] + ' ' + sample[1]) for sample in tqdm(train_data)]

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [99]:
for window_size in [2, 4, 6, 8, 10]:
    print(f'current window size: {window_size}')
    embeddings_trained = Word2Vec(words, vector_size=200, min_count=5, window=window_size, sg=1, workers=8).wv

    wv_ranking = []
    max_validation_examples = 100
    for i, line in enumerate(tqdm(validation_data[:max_validation_examples])):
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    print()

current window size: 2


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.310 | Hits@   1: 0.310
DCG@   5: 0.410 | Hits@   5: 0.510
DCG@  10: 0.427 | Hits@  10: 0.560
DCG@ 100: 0.467 | Hits@ 100: 0.760
DCG@ 500: 0.490 | Hits@ 500: 0.940
DCG@1000: 0.497 | Hits@1000: 1.000

current window size: 4


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.360 | Hits@   1: 0.360
DCG@   5: 0.459 | Hits@   5: 0.540
DCG@  10: 0.481 | Hits@  10: 0.610
DCG@ 100: 0.537 | Hits@ 100: 0.880
DCG@ 500: 0.548 | Hits@ 500: 0.970
DCG@1000: 0.551 | Hits@1000: 1.000

current window size: 10


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.390 | Hits@   1: 0.390
DCG@   5: 0.474 | Hits@   5: 0.560
DCG@  10: 0.503 | Hits@  10: 0.650
DCG@ 100: 0.555 | Hits@ 100: 0.900
DCG@ 500: 0.564 | Hits@ 500: 0.970
DCG@1000: 0.567 | Hits@1000: 1.000



### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:




1.   Лучшей токенизацией стала WordPunct токенизация (как и в принципе MyTokenizer на регулярных выражениях). Судя по всему, нормализация (лемматизация) слов в Pymorphy токенизаторе оказалась избыточна для данной задачи из-за специфики датасета (и большого обилия технических терминов в текстах со StackOverflow).
2.   В данном случае не помогает (по причинам, изложенным в пункте выше).
3.   Предобученные эмбеддинги в данном случае оказались чуть лучше по метрикам (что вполне может измениться, если взять больше данных для валидации и/или изменить параметры Word2Vec, попробовать CBOW, поменять min_count, и так далее). Из обученных на тренировочном датасете эмбеддингов наилучшее качество показали эмбеддинги с максимальным (равным 10) размером окна. Видимо в данном датасете слова имеют сильное влияние на контекст.
4.   Потому что Word2Vec давно уже не является SOTA методом построения эмбеддингов. В нем не учитывается факт, что слова могут иметь разные значения в разных контекстах, не используется информация о совместном появлении слов и т. д. К тому же, замерять cosine similarity между векторами слишком просто, можно использовать поверх нейронные сети для присвоения рангов предложениям.
5.   Использовать более современные техники построения эмбеддингов (FastText/BERT). Для ранжирования использовать другие техники (NN).

